In [1]:
# !pip install -U albumentations
import albumentations as A
import cv2
import os
import sys
import cv2
import random
import mediapipe as mp
import matplotlib.pyplot as plt
from tqdm import tqdm


mp_face_detection = mp.solutions.face_detection

In [2]:
# cvat labels to kitti txt files

# import xml.etree.ElementTree as ET
# import os
# import sys
# import shutil
# import cv2

# cvat_xml_file = "/data2/dsm-1507/dataset/Combined_csv/new_data/annotations_dec21.xml"

# kitti_annots_dest_path = "/data2/dsm-1507/dataset/Combined_csv/new_data/21_Dec_anno/"


# tree = ET.parse(cvat_xml_file)

# root = tree.getroot()

# for node in root.iter('image'):

#     image_dict = node.attrib

#     zero_annots = True

#     for elem in node.iter('box'):
#         bbox_dict = elem.attrib
#         if len(bbox_dict) != 0:
#             zero_annots =  False
#             break

#     if zero_annots==True:
#         continue

#     file_name = os.path.basename(image_dict["name"])

#     f = open(kitti_annots_dest_path + os.path.splitext(file_name)[0] + ".txt", "w")

#     for elem in node.iter('box'):
#         bbox_dict = elem.attrib
#         f.write("{0} 0.00 0 0.00 {1:.2f} {2:.2f} {3:.2f} {4:.2f} 0.00 0.00 0.00 0.00 0.00 0.00 0.00\n".format(bbox_dict["label"].lower(),float(bbox_dict["xtl"]),float(bbox_dict["ytl"]),float(bbox_dict["xbr"]),float(bbox_dict["ybr"])))
#     f.close()

In [59]:
# src_imgs_dir = '/data2/dsm-1507/dataset/Augmented_cellphone_cig/original_imgs_and_annos/smoking/smoking_3800'
# src_annots_dir  = '/data2/dsm-1507/dataset/Augmented_cellphone_cig/original_imgs_and_annos/smoking/original_annos'


# src_imgs_dir = '/data2/dsm-1507/dataset/Augmented_cellphone_cig/original_imgs_and_annos/cellphone/Cell_Phone_download'
# src_annots_dir  = '/data2/dsm-1507/dataset/Augmented_cellphone_cig/original_imgs_and_annos/cellphone/original_annos'


# src_imgs_dir = '/data2/dsm-1507/dataset/Combined_csv/new_data/21_Dec'
# src_annots_dir  = '/data2/dsm-1507/dataset/Combined_csv/new_data/21_Dec_anno'


# src_imgs_dir = '/data2/dsm-1507/dataset/Combined_csv/new_data/cellphone_new'
# src_annots_dir  = '/data2/dsm-1507/dataset/Combined_csv/new_data/cellphone_anno'


# src_imgs_dir = '/data2/dsm-1507/dataset/Combined_csv/new_data/cellphone_new_1_imgs'
# src_annots_dir  = '/data2/dsm-1507/dataset/Combined_csv/new_data/cellphone_new_1_anno'


# src_imgs_dir = '/data2/dsm-1507/dataset/Combined_csv/new_data/cig_new'
# src_annots_dir  = '/data2/dsm-1507/dataset/Combined_csv/new_data/cig_anno'


src_imgs_dir = '/data2/dsm-1507/dataset/Combined_csv/new_data/images_instagram_smoking'
src_annots_dir  = '/data2/dsm-1507/dataset/Combined_csv/new_data/images_instagram_smoking_anno'

In [60]:
# src_annots_dir  = "/data2/dsm-1507/dataset/Combined_csv/new_data/cig_anno"
# src_imgs_dir    = '/data2/dsm-1507/dataset/Combined_csv/new_data/cig_new'

dest_annots_dir ="/data2/dsm-1507/dataset/Combined_csv/single_aug/a"
dest_imgs_dir   = "/data2/dsm-1507/dataset/Combined_csv/single_aug/i"
# 

# dest_annots_dir ="/data2/dsm-1507/dataset/Combined_csv/with_face_annots/a_x"
# dest_imgs_dir   = "/data2/dsm-1507/dataset/Combined_csv/with_face_annots/i_x"

In [61]:
# Add face dets to annots
import os
import cv2
from tqdm import tqdm

anno_path = [os.path.join(src_annots_dir,x) for x in os.listdir(src_annots_dir)]
image_path = []
anno_path

for path in tqdm(anno_path):
    basename = os.path.basename(path)
#     print(basename)
    if os.path.exists((os.path.join(src_imgs_dir,f'{basename[:-4]}.jpg'))):
        im_p = f'{basename[:-4]}.jpg'
    elif os.path.exists((os.path.join(src_imgs_dir,f'{basename[:-4]}.jpg'))):
        im_p = f'{basename[:-4]}.jpg'
    elif os.path.exists((os.path.join(src_imgs_dir,f'{basename[:-4]}.jpeg'))):
        im_p = f'{basename[:-4]}.jpeg'
        
    if os.path.exists(os.path.join(src_imgs_dir,im_p)) is False:
        continue
    
    image_path.append(os.path.join(src_imgs_dir,im_p))
    
                     
# def face_crop_from_img(mp_face_detection,image,min_detection_confidence=0.55,prop = 1.5):
for i in tqdm(range(len(anno_path))):
    img = cv2.imread(image_path[i])
#     print(img.shape)


    face_loc = face_crop_from_img(mp_face_detection,img,min_detection_confidence=0.55,prop = 0.5)
#     print(face_loc)
    if face_loc is not None:
        f = open(anno_path[i], "a")
        f.write("{0} 0.00 0 0.00 {1:.2f} {2:.2f} {3:.2f} {4:.2f} 0.00 0.00 0.00 0.00 0.00 0.00 0.00\n".format('face',
                                                                                                              face_loc[0][0],
                                                                                                              face_loc[0][1],
                                                                                                              face_loc[1][0],
                                                                                                              face_loc[1][1]))

        f.close()

In [62]:
def face_crop_from_img(mp_face_detection,image,min_detection_confidence=0.55,prop = 1.5):
    def adjest_points(n,w,h):
            if n[0]<0:
                n[0]=0
            if n[1]<0:
                n[1]=0

            if n[0]>w:
                n[0]=w
            if n[1]>h:
                n[1]=h

            return([int(n[0]),int(n[1])])
    
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=min_detection_confidence) as face_detection:
        if type(image) == str:
            img = cv2.imread(image)
            h, w = img.shape[:2]
        else:
            img = image
            h, w = img.shape[:2]
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if not results.detections:
            return None

        for detection in results.detections:
            # NOSE_TIP = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.NOSE_TIP)
            # n = (int(NOSE_TIP.x*w),int(NOSE_TIP.y*h))

            # MOUTH_CENTER = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.MOUTH_CENTER)
            # m = (int(MOUTH_CENTER.x*w),int(MOUTH_CENTER.y*h))

            x = int(detection.location_data.relative_bounding_box.xmin*w)
            y = int(detection.location_data.relative_bounding_box.ymin*h)
            x1 = int((detection.location_data.relative_bounding_box.xmin+detection.location_data.relative_bounding_box.width)*w)
            y1 = int((detection.location_data.relative_bounding_box.ymin+detection.location_data.relative_bounding_box.height)*h)

            x,y = adjest_points([x,y],w,h)
            x1,y1 = adjest_points([x1,y1],w,h)
            #print(x,y,x1,y1)
            
            ox = (x+x1)*0.5
            oy = (y+y1)*0.5
            
            
            fx = ox - (x1-x)*prop
            fy = oy - (y1-y)*prop
            fx1 = ox + (x1-x)*prop
            fy1 = oy + (y1-y)*prop
            
            a = adjest_points([fx,fy],w,h)
            b = adjest_points([fx1,fy1],w,h)
            
        cropped = img[a[1]:b[1],a[0]:b[0]]
        data = a,b
    
    return data

In [63]:
def get_annos_from_txt(src_annots_dir,annotation_file):
    with open(os.path.join(src_annots_dir,annotation_file), "r") as ins:
        lines = [line.rstrip('\n') for line in ins]
        
        if lines==0:
            return None

        data = []
        for line in lines:
            coordinates = line.split()
            bbox = [coordinates[0],float(coordinates[4]),float(coordinates[5]),float(coordinates[6]),float(coordinates[7])]
            data.append(bbox)
        # f = open(dest_annots_dir + annotation_file, "w")   
        # 
    return data

In [64]:


def x1y1x2y2_to_xywh(bbox_list):
    x1 = bbox_list[0]
    y1 = bbox_list[1]
    x2 = bbox_list[2]
    y2 = bbox_list[3]
    
    w = x2-x1
    h = y2-y1
    
    return [int(x1),int(y1),int(w),int(h)]

def xywh_to_x1y1x2y2(bbox_list):
    x1 = bbox_list[0]
    y1 = bbox_list[1]
    w = bbox_list[2]
    h = bbox_list[3]
    
    x2 = x1+w
    y2 = y1+h
    
    return [int(x1),int(y1),int(x2),int(y2)]

def aug_im_box(image,bbox_list,transform):
    
    transformed = transform(image=image, bboxes=bbox_list)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    
    return transformed_image,transformed_bboxes
    

In [65]:
src_annots = sorted(os.listdir(src_annots_dir)) 
DO_AUG = True
ii = 15208 # no by which image is saved
no_aug = 5 # no of times the same image is augmented
for annotation_file in tqdm(src_annots):    
#     if ii>= 85669+100:
#         break

    try:
        
        for _ in range(no_aug):
            image_name = os.path.join(src_imgs_dir, os.path.basename(annotation_file)[:-3])
#             print('kk',image_name)
            if os.path.exists(image_name + "jpg"):
                image_name += "jpg"
            elif os.path.exists(image_name + "jpeg"):
                image_name += "jpeg"

            elif os.path.exists(image_name + "png"):
                image_name += "png"

            if os.path.exists(image_name) is False:
#                 print('image_name is False',ii)
                
                continue
            
            img = cv2.imread(image_name)
           
            


            anno_data = get_annos_from_txt(src_annots_dir,annotation_file)
            if anno_data is None:
#                 print('anno_data is None',ii)
                continue
#             print(face_data,anno_data,annotation_file)

            bbox_list = []
            for data in anno_data:
                x = []

                x.append(int(data[1]))
                x.append(int(data[2]))
                x.append(int(data[3]))
                x.append(int(data[4]))
                x = x1y1x2y2_to_xywh(x)

                x.append(data[0])
                bbox_list.append(x)

            transform = A.Compose([
                A.HorizontalFlip(p=0.0000001),
#                 A.RandomBrightnessContrast(p=0.2),
#                 A.Affine(p=0.5),
#                 A.RandomBrightnessContrast(p=0.2),
#                 A.CLAHE(),
#                 A.RGBShift(),
#                 A.ImageCompression(),
#                 A.GaussNoise(),
#                 A.RandomScale(),
                ], bbox_params=A.BboxParams(format='coco'))

            transformed_img, transformed_bbox = aug_im_box(img,bbox_list,transform)

            bbox_data = []
            for data in transformed_bbox :

                d = []
                d.append(data[-1])
                for x in xywh_to_x1y1x2y2(data[:-1]):
                    d.append(x)
                bbox_data.append(d)


            face_data = face_crop_from_img(mp_face_detection,transformed_img,min_detection_confidence=0.55,prop = 1.2)
            if face_data is None:
#                 print('face_data is None',ii)
                continue

            x1_list = []
            x1_list.append(face_data[0][0])
            for data in bbox_data:
                x1_list.append(data[1])

            y1_list = []
            y1_list.append(face_data[0][1])
            for data in bbox_data:
                y1_list.append(data[2])

            x2_list = []
            x2_list.append(face_data[1][0])
            for data in bbox_data:
                x2_list.append(data[3])

            y2_list = []
            y2_list.append(face_data[1][1])
            for data in bbox_data:
                y2_list.append(data[4])

            crop_cordi = [min(x1_list),min(y1_list),max(x2_list),max(y2_list)]

            scaled_annos = []
            for data in bbox_data:
                if data[0] != 'face':
                    x = []
                    x.append(data[0])
                    x.append(int(data[1]-crop_cordi[0]))
                    x.append(int(data[2]-crop_cordi[1]))
                    x.append(int(data[3]-crop_cordi[0]))
                    x.append(int(data[4]-crop_cordi[1]))
                    scaled_annos.append(x)

            a,b = face_data
            face_crp_img = transformed_img[a[1]:b[1],a[0]:b[0]].copy()



    #         print(bbox_data)


    #         for data in scaled_annos:
    #             img = cv2.rectangle(face_crp_img,(data[1],data[2]),(data[3],data[4]),(0,0,200), 1)
            ii+=1
            out_im_name = f'{dest_imgs_dir}/{ii}.jpg'
#             print(out_im_name)
            cv2.imwrite(out_im_name,face_crp_img)

            f = open(f'{dest_annots_dir}/{ii}.txt', "w")
            if DO_AUG is False:
                scaled_annos = anno_data
            for data in scaled_annos:
                if data[0] != 'face':
                    f.write("{0} 0.00 0 0.00 {1:.2f} {2:.2f} {3:.2f} {4:.2f} 0.00 0.00 0.00 0.00 0.00 0.00 0.00\n".format(data[0],
                                                                                                                  data[1],
                                                                                                                  data[2],
                                                                                                                  data[3],
                                                                                                                  data[4]))
            f.close()
            
    except:pass        
print(ii,'lllllll')

100%|███████████████████████████████████████████████████████████████████████| 4262/4262 [03:45<00:00, 18.93it/s]

19033 lllllll


In [67]:
# Ramdomly sample images for validation set
from random import sample
path = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_anno'
annos_path_list = os.listdir(path)

src = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_anno'
dst = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_anno_val'

x = sample(annos_path_list,(int(len(annos_path_list)*0.2)))


for file in x:
    os.rename(os.path.join(src,file), os.path.join(dst,file))

print(len(x))

3806


In [69]:

# convert rgb image to rrr image
import os
import cv2
import numpy as np
from tqdm import tqdm
src_path = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_imgs'
dst_path = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_imgs_rrr'
# /media/cheling/dataset2/dsm-1507/dataset/

for filename in tqdm(os.listdir(src_path)):
    image_path = os.path.join(src_path,filename)
    img = cv2.imread(image_path)
    
    r_im = img[:,:,2] #ir_img
    rrr_im = np.stack((r_im,)*3, axis=-1)#ir_img
    
    cv2.imwrite(f'{dst_path}/{filename[:-4]}.jpg',rrr_im)

100%|████████████████████████████████████████████████████████████████████| 23017/23017 [03:24<00:00, 112.60it/s]


In [71]:
# create csv file to train effidet models
import os
import sys
import imagesize
from tqdm import tqdm
# /media/cheling/dataset2/dsm-1507/dataset/Combined_csv/with_face_annots/aug_annos_val

f = open("/data2/dsm-1507/dataset/Combined_csv/single_aug/cig_hand_cellphone_jan5.csv","w")

annotations_dir = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_anno/'
val_annotations_dir = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_anno_val/'
x = sorted(os.listdir(annotations_dir))
y = sorted(os.listdir(val_annotations_dir))

from tqdm import tqdm
for annotation_file in (x):
    image_name = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_imgs/'+os.path.basename(annotation_file)[:-4] + '.jpg'
    with open(annotations_dir+annotation_file, "r") as ins:
            lines = [line.rstrip('\n') for line in open(annotations_dir+annotation_file)]
            if lines==0:
                continue
            width, height = imagesize.get(image_name)
            
            for line in lines:
                f.write("TRAIN,"+image_name)
                coordinates = line.split()
                bbox = [coordinates[0],int(float(coordinates[4])),int(float(coordinates[5])),int(float(coordinates[6])),int(float(coordinates[7]))]
                rect = [ bbox[1]/(width*1.0), bbox[2]/(height*1.0), bbox[3]/(width*1.0), bbox[4]/(height*1.0)]
                f.write(",{0},{1:.2f},{2:.2f},,,{3:.2f},{4:.2f},,\n".format(bbox[0],rect[0],rect[1],rect[2],rect[3]))

for annotation_file in y:
    #image_name = '/data2/dsm-training-1908/images_new_roi_gray_jpg/'+os.path.basename(annotation_file)[:-4] + '_gray.jpg'
    image_name = '/data2/dsm-1507/dataset/Combined_csv/single_aug/aug_imgs/'+os.path.basename(annotation_file)[:-4] + '.jpg'
    with open(val_annotations_dir+annotation_file, "r") as ins:
            lines = [line.rstrip('\n') for line in open(val_annotations_dir+annotation_file)]
            if lines==0:
                continue
            width, height = imagesize.get(image_name)
            for line in lines:
                f.write("VALIDATION,"+image_name)
                coordinates = line.split()
                bbox = [coordinates[0],int(float(coordinates[4])),int(float(coordinates[5])),int(float(coordinates[6])),int(float(coordinates[7]))]
                rect = [ bbox[1]/(width*1.0), bbox[2]/(height*1.0), bbox[3]/(width*1.0), bbox[4]/(height*1.0)]
                f.write(",{0},{1:.2f},{2:.2f},,,{3:.2f},{4:.2f},,\n".format(bbox[0],rect[0],rect[1],rect[2],rect[3]))